In [1]:
import pandas as pd

In [3]:
user_data = pd.read_csv('user_data.csv', low_memory=False)
subscriptions = pd.read_csv('subscriptions.csv', low_memory=False)
marketing_spend = pd.read_csv('marketing_spend.csv', low_memory=False)

In [5]:
subscriptions['customer_account_id'] = pd.to_numeric(subscriptions['customer_account_id'],errors='coerce').astype('Int64')

In [7]:
subs_success = subscriptions[subscriptions['invoice_status'] == 'success']

In [9]:
total_rev_per_cust = (subs_success.groupby('customer_account_id')['order_amount'].sum())

In [11]:
mean_total_revenue = total_rev_per_cust.mean()

In [13]:
print(f"Mean total revenue per customer: ${mean_total_revenue:.2f}")

Mean total revenue per customer: $111.03


In [15]:
user_data['purchase_date_utc'] = pd.to_datetime(user_data['purchase_date_utc'], errors='coerce')
subscriptions['order_created_at'] = pd.to_datetime(subscriptions['order_created_at'], errors='coerce')
user_data['customer_account_id']  = pd.to_numeric(user_data['customer_account_id'], errors='coerce').astype('Int64')
subscriptions['customer_account_id'] = pd.to_numeric(subscriptions['customer_account_id'], errors='coerce').astype('Int64')

In [17]:
cohort = user_data[(user_data['purchase_date_utc'].dt.year == 2024) &(user_data['purchase_date_utc'].dt.month == 7)]
cohort_ids = cohort['customer_account_id'].dropna().astype(int).unique()

In [19]:
subs_ok = subscriptions[(subscriptions['invoice_status'] == 'success') &(subscriptions['customer_account_id'].isin(cohort_ids))].copy()

In [21]:
subs_ok = subs_ok.merge(cohort[['customer_account_id', 'purchase_date_utc']],on='customer_account_id', how='left')
subs_ok['days_since'] = (subs_ok['order_created_at'] - subs_ok['purchase_date_utc']).dt.days

In [23]:
revenue_3m  = subs_ok[subs_ok['days_since'] <=  90]['order_amount'].sum()
revenue_6m  = subs_ok[subs_ok['days_since'] <= 180]['order_amount'].sum()
revenue_12m = subs_ok[subs_ok['days_since'] <= 365]['order_amount'].sum()

In [25]:
print(f"3 months:  ${revenue_3m:,.2f}")
print(f"6 months:  ${revenue_6m:,.2f}")
print(f"12 months: ${revenue_12m:,.2f}")

3 months:  $2,198,551.64
6 months:  $2,837,187.83
12 months: $3,462,383.19


In [27]:
user_data['purchase_date_utc'] = pd.to_datetime(user_data['purchase_date_utc'], errors='coerce')
subscriptions['order_created_at'] = pd.to_datetime(subscriptions['order_created_at'], errors='coerce')
user_data['customer_account_id'] = pd.to_numeric(user_data['customer_account_id'], errors='coerce').astype('Int64')
subscriptions['customer_account_id'] = pd.to_numeric(subscriptions['customer_account_id'], errors='coerce').astype('Int64')

In [29]:
cohort = user_data[(user_data['purchase_date_utc'].dt.year == 2024) &(user_data['purchase_date_utc'].dt.month == 7)]
cohort_ids = cohort['customer_account_id'].dropna().astype(int).unique()

In [31]:
subs_ok = subscriptions[subscriptions['invoice_status'] == 'success']
subs_cohort = subs_ok[subs_ok['customer_account_id'].isin(cohort_ids)].copy()

In [33]:
subs_cohort = subs_cohort.merge(cohort[['customer_account_id','purchase_date_utc']],on='customer_account_id', how='left')
subs_cohort['days_since'] = (subs_cohort['order_created_at'] - subs_cohort['purchase_date_utc']).dt.days

In [35]:
def avg_ltv(days_limit):
    temp = subs_cohort[(subs_cohort['days_since'] >= 0) &(subs_cohort['days_since'] <= days_limit)]
    per_user = temp.groupby('customer_account_id')['order_amount'].sum()
    per_user = per_user.reindex(cohort_ids, fill_value=0)
    return per_user.mean()

ltv_3m  = avg_ltv( 90)
ltv_6m  = avg_ltv(180)
ltv_12m = avg_ltv(365)

print("Средний  кумулятивный доход на пользователя:")
print(f"  3 мес:  ${ltv_3m: .2f}")
print(f"  6 мес:  ${ltv_6m: .2f}")
print(f" 12 мес:  ${ltv_12m: .2f}")

Средний  кумулятивный доход на пользователя:
  3 мес:  $ 68.66
  6 мес:  $ 88.62
 12 мес:  $ 108.15
